In [2]:
# Aurthor: Elihu Essien-Thomspon
# Program Objective: Experiment 3
# Finding the best PSO algortihm
# representative to use for
# the final Experiment.


import numpy as np
import random
import math

In [3]:
# Returns the distance between two 
# points(x, y) on the map

def cityDistance(map, cityID1, cityID2):
    x = abs(map[cityID1][0] - map[cityID2][0])
    y = abs(map[cityID1][1] - map[cityID2][1])
    
    # pythagoras
    dist = math.sqrt(x**2 + y**2)
    return(dist)

In [14]:
# Reading in map data from text file
# and returning a distance matrix to use
# for calculations
def getMap(size,id):
    cities = np.empty((0,2), dtype=int)      # initialize empty 2D vector list
    
    # open file
    with open(f"C:/Users/C14460702/Dissertation/Data/Maps/Size - {size}/map{id}.txt", 'r') as f:
        # extract vector list data from the comma separated string list data
        for line in f:
            line = line.strip('\n')    # remove '\n'
            vec = line.split(', ')     # split by ', '
            
            # cast co-ordinate vector as integers and insert into list 
            cities = np.append(cities, [[ int(vec[0]), int(vec[1]) ]], axis=0)
    f.close()
    
    # construct distance matrix
    mapSize = len(cities)
    distanceTable = np.zeros((mapSize,mapSize))
    for row in range(mapSize):
        for col in range(mapSize):
            distanceTable[row][col] = cityDistance(cities, row, col)
    
    return (distanceTable)    # return vector list

In [5]:
# maps a variable exising on one range to another range
def mapRanges(value, currlow, currHigh, newLow, newHigh):
    # extract ranges
    currRange = currHigh - currlow
    newRange = newHigh - newLow
    
    # percentage of range that the value takes up
    valPercent = float(value - currlow) / float(currRange)
    
    # translate percentage to new range
    newVal = (newRange * valPercent) + newLow
    
    return(newVal)

In [6]:
# This function takes in a 1D array of choices
# where the each member's ID represents the choice
# and each member's value represents the percent (0-1)
# that member has of comming true. The algorithm then
# selects a winner by performing a weighted random choice

def weightedChoice(percentages):
    choiceIDs = [i for i in range(len(percentages))]
    percentages = [mapRanges(i, 0, 1, 0, 100) for i in percentages]
    
    # choose winner
    winnerID = random.choices(choiceIDs, weights = percentages, k = 1)[0]
    return (winnerID)

In [7]:
# Function takes in a route list
# and a distnce matrix to be used
# to score the route given

def routeScore(route, distanceTable):
    
    dist = 0;
    
    # aggregate distances between cities
    # on the list given
    for i in range(len(route)):
        cur = route[i]
        nxt = route[(i+1) % len(route)]  # includes loop back to start
        dist += distanceTable[cur][nxt]  # get the distances from the matrix
        
    
    # the smaller the distance, the higher the score
    return (1/dist)

In [8]:
def Ant(startingPosition, mapSize, distanceTable, pheromoneTable, alpha, beta, localPhero = None):
    position = startingPosition
    visited = []
    q0 = 1
    
    
    while (len(visited) < mapSize-1):
        # add current city possition to the visited list
        visited += [position]
        
        # next city options
        ChoiceIDs = [i for i in range(mapSize) if i not in visited]

        
        # ACS
        if(localPhero != None):
            q0 = 0.9
        
        
        if( random.random() < q0): # Stochastic Choice Rule
            # calculate choice weights
            choiceAttractiveness = [(pheromoneTable[position][cityID] ** alpha) * ((1/distanceTable[position][cityID]) ** beta) for cityID in ChoiceIDs]
            total = sum(choiceAttractiveness)
            weigths = [i/total for i in choiceAttractiveness]
            
            # move to next city
            position = ChoiceIDs[ weightedChoice(weigths) ]
            
        else: # Deterministic Choice Rule
            choiceAttractiveness = [(pheromoneTable[position][cityID] ** alpha) * ((1/distanceTable[position][cityID]) ** beta) for cityID in ChoiceIDs]
            
            best = [0,0]  # [id, score]
            # find the most attractive
            for i in range(len(choiceAttractiveness)):
                if(choiceAttractiveness[i] > best[1]):
                    best = [i, choiceAttractiveness[i]]
            
            # move to next city
            position = ChoiceIDs[ best[0] ]
        
        
        # AS local pheromone update
        if(localPhero != None):
            pheromoneTable[visited[len(visited)-1]][position] += localPhero
            pheromoneTable[position][visited[len(visited)-1]] += localPhero
        
            
    # add the final position to the list
    visited += [position]
    
    
    if(localPhero != None):
        return(visited, pheromoneTable)
    return(visited)

In [9]:
def clamp(Num, Min, Max):
    return (max(min(Num, Max), Min))

In [10]:
# Pheramone update function for the Ant System (AS)
# for the TSP allows all ants to update the pheramone
# table adding pheramones to thier traveled paths 
# relative to the overall score of thier chosen route
def PheromoneUpdate(routes, scores, pheromoneTable, TMax, TMin):
    
    for i in range(len(routes)):
        route = routes[i]
        
        # update pheramone levels for the paths taken
        # with the overall score of the route
        for j in range(len(route)):
            score = clamp(scores[i], TMin, TMax) # Max-Min Clamp
            cur = route[j]
            nxt = route[(j+1) % len(route)]
            
            # update each edge with the score for that route
            pheromoneTable[cur][nxt] += score
            pheromoneTable[nxt][cur] += score
    
    return (pheromoneTable)

In [11]:
def ACO(distanceTable, maxIterations, popSize, alpha, beta, evaporationRate, Pbest = 0, MaxMin = False, ACS = False):
    # Initialize pheramone matrix to a trivially small number
    # to avoid divide by zero
    mapSize = len(distanceTable)
    pheromoneTable = [[0.0000000001 for col in range(mapSize)] for row in range(mapSize)]
    TMax, TMin = math.inf, 0
    localPhero = 0
    
    # records
    GBestScore_PerItreation = []
    Gbest = [[], 0]
    for j in range(maxIterations):
        
        
        # Scalable algorithm configurations
        if(j > 0):
            if(MaxMin):
                TMax = 1/(1-evaporationRate) * 1/Gbest[1]
                if(Pbest == 0):
                    TMin = 0
                else:
                    TMin = (TMax * (1-math.pow(Pbest, 1/mapSize))) / ((mapSize/2 - 1) * math.pow(Pbest, 1/mapSize))
            elif(ACS):
                localPhero = evaporationRate * math.pow((mapSize * 1/Gbest[1]), -1)
        
        
        
        # Specialized Initialization
        if(j == 1):
            if(MaxMin):
                for row in range(mapSize):
                    for col in range(mapSize):
                        pheromoneTable[row][col] = TMax
            elif(ACS):
                for row in range(mapSize):
                    for col in range(mapSize):
                        pheromoneTable[row][col] = math.pow((mapSize * 1/Gbest[1]), -1)
            
        
        # Step 1 - Ants traverse map
        # run ants accross map to collect list of solutions
        # each ant starting at a random position
        routes = []
        scores = []
        Ibest = [[], 0]   # for Max-Min
        
        # get itteration data
        for member in range(popSize):
            # run ants starting at random positions
            if(ACS):
                route, pheromoneTable = Ant(random.choice(range(mapSize)), mapSize, distanceTable, pheromoneTable, alpha, beta, localPhero)
            else:
                route = Ant(random.choice(range(mapSize)), mapSize, distanceTable, pheromoneTable, alpha, beta)
            routes += [route]
            scores += [routeScore(route, distanceTable)]
        
        # Step 2 - Update data based on results
        # find best score for this iteration
        for i in range(len(scores)):
            if(scores[i] > Ibest[1]):
                Ibest = [routes[i], scores[i]]
        
        
        if(Ibest[1] > Gbest[1]):
            Gbest = Ibest
        GBestScore_PerItreation += [Gbest[1]]
        
        
        # Step 3 - Pheramone Update
        pheramoneTable = [[(1-evaporationRate) * col for col in row] for row in pheromoneTable] # evaporation
        
        # deposit
        if(MaxMin):
            pheramoneTable = PheromoneUpdate([Ibest[0]], [Ibest[1]], [i for i in pheromoneTable], TMax, TMin)
        else:
            pheramoneTable = PheromoneUpdate(routes, scores, [i for i in pheromoneTable], math.inf, 0)
            
        
        
    
    
    return(GBestScore_PerItreation)

In [11]:
# Algorithm settings
popSize = 50            # Number of chromosomes to use
maxIterations = 100    # Maximum number of generations
alpha = 1
beta = 2
evaporationRate = 0.9


# Traveling Salesman Problem Settings
mapSize = 10        # cities per map

In [15]:
# Experiment 2(A) - Max-Min Configurations
# finding configuration for the Pbest

mapCounter = 0      # itteration counter

while (True):
    try:
        # get map
        distanceTable = getMap(mapSize,mapCounter)
        
        for i in range(5):
            Pbest = 0
            if (i > 0):
                Pbest = 5/(10**i)
                
            with open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment 2/A/MMAS({Pbest}).txt", 'a') as f:
                data = ACO(distanceTable, maxIterations, popSize, alpha, beta, evaporationRate, Pbest, True)
                f.write(str(data))
        
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1
    except FileNotFoundError:
        break

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish

In [20]:
# Experiment 2B - AS vs MMAS

#Settings from experiement 2
Pbest = 0.0005

mapCounter = 0      # itteration counter
while (True):
    try:
        # get map
        distanceTable = getMap(mapSize,mapCounter)
        
        with open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment 2/B/ACS.txt", 'a') as f:
            data = ACO(distanceTable, maxIterations, popSize, alpha, beta, evaporationRate, Pbest, False, True)
            f.write(str(data))
        
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1
    except FileNotFoundError:
        break

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish

In [17]:
# Collecting best data for final Experiment 4
# Best ACO settings found
evaporationRate = 0.9
alpha = 1
beta = 2


# Traveling Salesman Problem Settings
numCities = 50        # cities per map
mapCounter = 0      # itteration counter
popSize = 50
maxIterations = 200


while(True):
    try:
        distanceTable =  getMap(numCities,mapCounter)
        
        with open (f"C:/Users/C14460702/Dissertation/Data/Results/Experiment 4/Size - {numCities}/ACO.txt", "a") as f:
            data = ACO(distanceTable, maxIterations, popSize, alpha, beta, evaporationRate)
            f.writelines(str(data))
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1
    except FileNotFoundError:
        break

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish